[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/eirasf/GCED-AA2/blob/main/lab6/lab6-parte2.ipynb)
# Práctica 6: Redes neuronales convolucionales - Complicando la CNN


### Pre-requisitos. Instalar paquetes

Para la primera parte de este Laboratorio 6 necesitaremos TensorFlow y TensorFlow-Datasets. Además, como habitualmente, fijaremos la semilla aleatoria para asegurar la reproducibilidad de los experimentos.

In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds

#Fijamos la semilla para poder reproducir los resultados
import os
import numpy as np
import random
seed=1234567
os.environ['PYTHONHASHSEED']=str(seed)
tf.random.set_seed(seed)
np.random.seed(seed)
random.seed(seed)

Además, cargamos también APIs que vamos a emplear para que el código quede más legible

In [ ]:
#API de Keras, modelo Sequential y la capa Dense 
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense 
#Para mostrar gráficas
from matplotlib import pyplot

### Carga del conjunto de datos

En esta ocasión trabajaremos con el conjunto de imágenes *cifar10*.

In [ ]:
import tensorflow_datasets as tfds

# El parámetro with_info=True nos permite acceder a información sobre el dataset
# Carga el conjunto de datos mnist. Usaremos el primer 80% de la partición train para ds_train y el 20% restante para ds_val. ds_test tomará la partición test.
(ds_train, ds_test, ds_val), ds_info = tfds.load(..., with_info=True, as_supervised=True)


# En dicha información se encuentran los nombres de las clases y las dimensiones de las imágenes
NUM_CLASSES = ds_info.features['label'].num_classes
nombres_clases = ds_info.features['label'].names
dimensiones = ds_info.features['image'].shape

# Para comprobar que se ha cargado tomamos un elemento y lo mostramos
ej_imagen, ej_etiqueta = next(iter(ds_train.take(1)))
pyplot.imshow(ej_imagen)
pyplot.xlabel(nombres_clases[ej_etiqueta.numpy()])
pyplot.show()


## Preprocesado de los datos


Nuevamente deberíamos convertir las etiquetas suministradas a codificación one_hot, pero vamos a tomar otra alternativa. Seguiremos prediciendo un vector con una componente por clase, pero mantendremos la etiqueta numérica y luego utilizaremos funciones de pérdida y métricas adaptadas a esta circunstancia. No obstante, seguimos teniendo que escalar los píxeles de la imagen al rango \[0,1\].

In [ ]:
ds_train = ...
ds_val = ...
ds_test = ...

## Creando el modelo

Declara ahora un modelo convolucional con la siguiente arquitectura:
 1. [Convolución 2D](https://keras.io/api/layers/convolution_layers/convolution2d/) de 32 filtros y stride 3, con activación ReLU
 1. [Pooling 2D](https://keras.io/api/layers/pooling_layers/max_pooling2d/) tomando el máximo de cada grupo de 2x2
  1. [Convolución 2D](https://keras.io/api/layers/convolution_layers/convolution2d/) de 64 filtros y stride 3, con activación ReLU
 1. [Pooling 2D](https://keras.io/api/layers/pooling_layers/max_pooling2d/) tomando el máximo de cada grupo de 2x2
 1. [Convolución 2D](https://keras.io/api/layers/convolution_layers/convolution2d/) de 64 filtros y stride 3, con activación ReLU
 1. Capa Densa (requiere aplanado previo) de 64 unidades y activación ReLU
 1. Capa de salida
 
Ejecuta la siguiente celda y repite el compilado, entrenamiento y verificaciones posteriores para observar la diferencia.

In [ ]:
# TODO - Crea el modelo descrito
model = ...

#Construimos el modelo y mostramos 
model.build()
print(model.summary())

# VERIFICACIÓN
assert model.count_params()==122570, 'Revisa la arquitectura de tu modelo'

### Entrenamiento del modelo
Vamos a establecer la función de pérdida, el optimizador (Adam con el LR por defecto).

Usaremos la entropía cruzada categórica (de nuevo con logits y utilizando la versión Sparse ya que las etiquetas son enteros).

Como métrica repetiremos con la precisión categórica, pero también en su versión *sparse*.

In [ ]:
#TODO - Compila el modelo con los parámetros indicados
model.compile(...)

Como siempre, entrenaremos el modelo usando `model.fit`. Para ello, previamente debemos indicar a nuestro dataset que haga lotes de 128 elementos. Le indicaremos también que baraje los datos utilizando un buffer de 5 veces el tamaño de lote. La aleatorización debe hacerse antes de la partición en lotes, para que se aleatoricen los elementos y no los lotes.

In [ ]:
# TODO - Baraja y trocea los datasets en lotes.
ds_train_batch = ...
ds_val_batch = ...

# TODO - Entrena el modelo. Con 20 epochs será suficiente.
# Haz que nos ofrezca también las mediciones de pérdida y precisión sobre el conjunto de validación,
# para saber si el modelo está sobreajustando.
history = model.fit(...)

In [ ]:
# TODO - Muestra las gráficas de pérdida y precisión para entrenamiento y validación

### Verificación del rendimiento
Aprovecharemos el conjunto de test para comprobar la capacidad de generalización de nuestro modelo.

In [ ]:
# TODO - Evalúa el modelo sobre el conjunto de test. Previamente deberás hacer lotes con el conjunto de test.
print("Evaluación sobre el conjunto TEST:")
ds_test_batch = ...
...

Si todo ha ido bien, deberías haber obtenido una preción en test del 60%, lo cual no es desdeñable para una red sencilla.

## Mejora del rendimiento
Vuelve sobre la arquitectura del modelo e incluye capas de [BatchNormalization](https://keras.io/api/layers/normalization_layers/batch_normalization/) después de cada capa convolucional. Esto hará que las salidas de esa capa se estandaricen para que sigan una distribución N(0,1) (la operación de estandarización se incluye en el grafo y, por tanto, el cómputo de los gradientes). El efecto de esta capa es que se evitará que el gradiente del lote tenga una gran componente solo dedicada a acercar las salidas a la media/desviaición de las muestras en cada capa. En consecuencia, el aprendizaje se acelera.

### Ejercicios
 - Repite el entrenamiento con la nueva arquitectura. ¿Qué efecto has notado? ¿Puedes mejorar el rendimiento en test utilizando conceptos vistos en Laboratorios anteriores?
 - Prueba distintas arquitecturas e intenta mejorar el rendimiento en test.